In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def extra_trees_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_estimators=100,
    max_depth=None,
    random_state=42,
    use_cv=False,
    cv_params=None,
    cv=5
):
    """
    Extra Trees regression for time series forecasting with:
    - Extremely Randomized Trees (ExtraTrees)
    - Time-series aware cross-validation
    - Feature importance analysis
    """
    # 1. Directory setup
    os.makedirs('ExtraTreesRegression', exist_ok=True)
    
    # 2. Stationarity handling
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 3. Create lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 4. Temporal split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 5. Model training with optional CV
    if use_cv:
        param_grid = cv_params or {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5],
            'bootstrap': [True, False]
        }
        tscv = TimeSeriesSplit(n_splits=cv)
        grid_search = GridSearchCV(
            ExtraTreesRegressor(random_state=random_state),
            param_grid,
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        model = ExtraTreesRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state
        )
        model.fit(X_train, y_train)
        best_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth
        }

    # 6. Forecasting
    pred_test = model.predict(X_test)

    # 7. Inverse differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 8. Save forecasts
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test
    })
    forecast_csv_path = os.path.join('ExtraTreesRegression', 'et_forecasts.csv')
    forecast_df.to_csv(forecast_csv_path, mode='a', header=not os.path.exists(forecast_csv_path), index=False)

    # 9. Calculate metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'best_params': best_params,
        'differencing': d,
        'feature_importances': model.feature_importances_.tolist()
    }
    metrics_df = pd.DataFrame([metrics_data])
    metrics_csv_path = os.path.join('ExtraTreesRegression', 'et_metrics.csv')
    metrics_df.to_csv(metrics_csv_path, mode='a', header=not os.path.exists(metrics_csv_path), index=False)

    # 10. Visualization
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast'], label='ET Forecast', color='darkred')
    plt.title(f'Extra Trees Forecast for {district_name}\nRMSE: {rmse:.2f}, Trees: {model.n_estimators}')
    plt.legend()
    plt.savefig(os.path.join('ExtraTreesRegression', f'et_forecast_{district_name}.png'))
    plt.close()

    # Feature importance plot
    features = X_train.columns
    importances = model.feature_importances_
    plt.figure(figsize=(10, 6))
    plt.barh(features, importances, color='darkred')
    plt.title(f'Feature Importances - {district_name}')
    plt.savefig(os.path.join('ExtraTreesRegression', f'et_importances_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'feature_importances': dict(zip(features, importances)),
        'model_params': model.get_params()
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I40"].asfreq('MS')
        
        results = extra_trees_regression_lags_only(
            ts,
            district,
            max_lags=6,
            n_estimators=200,
            max_depth=15
        )

        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district       rmse  r_squared                             best_params  \
0  AHMEDNAGAR  18.156512        1.0  {'n_estimators': 200, 'max_depth': 15}   

   differencing                                feature_importances  
0             0  [0.25219019738286247, 0.17598307238294772, 0.1...  

=== Forecast Data ===
     district       date  actual  forecast
0  AHMEDNAGAR 2020-07-01    40.0    37.820
1  AHMEDNAGAR 2020-08-01    55.0    41.525
2  AHMEDNAGAR 2020-09-01    36.0    45.590
3  AHMEDNAGAR 2020-10-01    82.0    34.175
4  AHMEDNAGAR 2020-11-01    49.0    48.035
=== Metrics ===
  district       rmse  r_squared                             best_params  \
0    AKOLA  11.288508        1.0  {'n_estimators': 200, 'max_depth': 15}   

   differencing                                feature_importances  
0             0  [0.20474690784124616, 0.18998500342934682, 0.1...  

=== Forecast Data ===
  district       date  actual  forecast
0    AKOLA 2020-07-01    34.0    45.

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual  forecast
 0  MUMBAI SUBURBAN 2021-03-01      28     23.86,
 'metrics_df':           district  rmse  r_squared                             best_params  \
 0  MUMBAI SUBURBAN  4.14        1.0  {'n_estimators': 200, 'max_depth': 15}   
 
    differencing                                feature_importances  
 0             1  [0.4929735761471163, 0.12071148009284052, 0.05...  ,
 'feature_importances': {'lag_1': 0.4929735761471163,
  'lag_2': 0.12071148009284052,
  'lag_3': 0.05158721656846998,
  'lag_4': 0.04469201928227106,
  'lag_5': 0.0425209783967149,
  'lag_6': 0.2475147295125872},
 'model_params': {'bootstrap': False,
  'ccp_alpha': 0.0,
  'criterion': 'squared_error',
  'max_depth': 15,
  'max_features': 1.0,
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
